# Hubbard Model: A Tunable Model of Electron Correlation
**Evangeslista Group Rotation Project**

Brian Zhao, 10 Oct 2022

We first consider the simplest case, an open, one-dimensional Hubbard model. For an $N$-site $1$-D Hubbard model, we represent a configuration as
$$
|\mathbf{\text{n}}\rangle = |\underbrace{k_{1\uparrow}k_{2\uparrow}k_{3\uparrow}\dots k_{N\uparrow}}_{\text{alpha}}\underbrace{k_{1\downarrow}k_{2\downarrow}k_{3\downarrow}\dots k_{N\downarrow}}_{\text{beta}}\rangle
$$
where $k_i$ is the occupatation number of the $i$-th spinorbital.

In [46]:
import numpy as np
import scipy.special
from IPython.display import display, Math

class HubbardLattice1D:
    def __init__(self, nsites, nalpha, nbeta, t, u):
        self.nsites = nsites
        self.nalpha = nalpha
        self.nbeta = nbeta
        self.t = t
        self.u = u
        self.ms = (nalpha-nbeta)/2.0
        
        self.nalpha_combinations = None
        self.nbeta_combinations = None
        self.ncomb = None
        self.mem_reqd = None
        self.alpha_strings = None
        self.beta_strings = None
        self.tot_strings = None
        
        # Unlikely do be doing more than 32 sites for now..
        #self.bit_string_len = int(np.ceil(self.nsites*2/64))

    def enumerate_states(self):
        """
        Enumerates all states of hl, only conserving Ms symmetry.
        """
        self.nalpha_combinations = int(scipy.special.binom(self.nsites, self.nalpha))
        self.nbeta_combinations = int(scipy.special.binom(self.nsites, self.nbeta))
        self.ncomb = self.nalpha_combinations * self.nbeta_combinations
        self.mem_reqd = self.ncomb*8/1e3 # We use 64-bit integers

        print(f"""There are {self.nalpha_combinations:d} alpha combinations,
{self.nbeta_combinations:d} beta combinations,
making a total of {self.ncomb:d} combinations,
requiring a memory of {self.mem_reqd:.2f} kB.""")

        self.alpha_strings = np.zeros(self.nalpha_combinations, dtype='int64')
        self.beta_strings = np.zeros(self.nbeta_combinations, dtype='int64')
        self.tot_strings = np.zeros(self.ncomb, dtype='int64')

        comb = list(range(self.nalpha))
        self.alpha_strings[0] = occ_list_to_bit_string(comb)
        for i in range(1,self.nalpha_combinations):
            ierr, comb = next_comb(self.nsites, self.nalpha, comb)
            if (ierr == 1):
                print('Too many combinations! Breaking.')
                break
            self.alpha_strings[i] = occ_list_to_bit_string(comb)

        comb = list(range(self.nbeta))
        self.beta_strings[0] = occ_list_to_bit_string(comb)
        for i in range(1,self.nbeta_combinations):
            ierr, comb = next_comb(self.nsites, self.nbeta, comb)
            if (ierr == 1):
                print('Too many combinations! Breaking.')
                break
            self.beta_strings[i] = occ_list_to_bit_string(comb)

        istring = 0
        for ialpha in range(self.nalpha_combinations):
            for ibeta in range(self.nbeta_combinations):
                self.tot_strings[istring] = ((self.beta_strings[ibeta])<<self.nsites) | (self.alpha_strings[ialpha])
                istring += 1
                
    def bit_string_to_occ_vec(self, bstring):
        ov_alpha = np.zeros(self.nsites, dtype='int32')
        ov_beta = np.zeros(self.nsites, dtype='int32')

        for i in range(self.nsites):
            ov_alpha[i] += (bstring>>i) & 1
            ov_beta[i] += (bstring>>(i+self.nsites)) & 1

        return ov_alpha, ov_beta, (ov_alpha+ov_beta)


    def pretty_print_state(self, bstring):
        ov_alpha,ov_beta,_ = self.bit_string_to_occ_vec(bstring)
        string = '$|'
        for i in range(self.nsites):
            if ov_alpha[i] == 0 and ov_beta[i] == 0:
                string += '-'
            elif ov_alpha[i] == 1 and ov_beta[i] == 0:
                string += '\\uparrow'
            elif ov_alpha[i] == 0 and ov_beta[i] == 1:
                string += '\\downarrow'
            elif ov_alpha[i] == 1 and ov_beta[i] == 1:
                string += '\\uparrow\\downarrow'

            if i != self.nsites-1:
                string += ','
            else:
                string += '\\rangle$'

        display(Math(string))

    def pretty_print_states(self):
        print(f'There are {self.ncomb:d} states with {self.nalpha:d} alpha electrons and {self.nbeta:d} beta electrons in {self.nsites:d} sites.')
        for i in range(len(self.tot_strings)):
            self.pretty_print_state(self.tot_strings[i])
            
    def get_hmatel(self):
        pass
            
def get_excitation_level(f1, f2):
    return int(count_set_bits(f1^f2)/2)

def count_set_bits(f):
    return int(bin(f).count('1'))

def set_bit(f, bit_loc):
    """
    bit_loc is zero-indexed
    """
    return f | 1<<bit_loc

def clear_bit(f, bit_loc):
    """
    bit_loc is zero-indexed
    """
    return f & ~(1<<bit_loc)

def next_comb(n, k, comb):
    i = k-1
    comb[i] += 1
    
    while (comb[i] >= n - k + 1 + i):
        i -= 1
        if (i < 0):
            break
        comb[i] += 1
    
    if (comb[0] > n-k):
        ierr = 1
    else:
        ierr = 0
        for j in range(i+1, k):
            comb[j] = comb[j-1] + 1
            
    return ierr, comb

def occ_list_to_bit_string(occ_list):
    bstring = 0
    for i in range(len(occ_list)):
        bstring = set_bit(bstring, occ_list[i])
    return bstring

def creop(bit_string, spinorb):
    # If the bit is set then sgn = 0
    if ((bit_string>>(spinorb)) & 1):
        sgn = 0
    else:
        test_string = 0
        for i in range(spinorb):
            test_string = set_bit(test_string, i)
        sgn = (-1)**(count_set_bits(bit_string & test_string))
        bit_string = set_bit(bit_string, spinorb)
        
    return (sgn, bit_string)

def annop(bit_string, spinorb):
    if ((bit_string>>(spinorb)) & 0):
        sgn = 0
    else:
        test_string = 0
        for i in range(spinorb):
            test_string = set_bit(test_string, i)
        sgn = (-1)**(count_set_bits(bit_string & test_string))
        bit_string = clear_bit(bit_string, spinorb)
        
    return (sgn, bit_string)

## Testing ground below

In [47]:
annop(3,1)

(-1, 1)

In [34]:
get_excitation_level(15,51)

2

In [37]:
set_bit(15,4)

31

In [23]:
hl = HubbardLattice1D(4, 2, 2, 1, 0)
hl.enumerate_states()
#hl.pretty_print_states()

There are 6 alpha combinations,
6 beta combinations,
making a total of 36 combinations,
requiring a memory of 0.29 kB.


In [39]:
occ_list_to_bit_string([0,1,2,4])

23

In [24]:
ova, ovb, ovt = bit_string_to_occ_vec(hl, 83)

In [33]:
string = pretty_print_state(hl, ova, ovb)

<IPython.core.display.Math object>

In [31]:
display(Math(string))

<IPython.core.display.Math object>

In [28]:
ovt

array([2, 1, 1, 0], dtype=int32)

In [31]:
comb = list(range(4))
print(comb)
ncomb = 1
while True:
    ierr,comb = next_comb(8,4,comb)
    if (ierr == 1):
        break
    print(comb)
    ncomb += 1
print(f'ncomb = {ncomb}')

[0, 1, 2, 3]
[0, 1, 2, 4]
[0, 1, 2, 5]
[0, 1, 2, 6]
[0, 1, 2, 7]
[0, 1, 3, 4]
[0, 1, 3, 5]
[0, 1, 3, 6]
[0, 1, 3, 7]
[0, 1, 4, 5]
[0, 1, 4, 6]
[0, 1, 4, 7]
[0, 1, 5, 6]
[0, 1, 5, 7]
[0, 1, 6, 7]
[0, 2, 3, 4]
[0, 2, 3, 5]
[0, 2, 3, 6]
[0, 2, 3, 7]
[0, 2, 4, 5]
[0, 2, 4, 6]
[0, 2, 4, 7]
[0, 2, 5, 6]
[0, 2, 5, 7]
[0, 2, 6, 7]
[0, 3, 4, 5]
[0, 3, 4, 6]
[0, 3, 4, 7]
[0, 3, 5, 6]
[0, 3, 5, 7]
[0, 3, 6, 7]
[0, 4, 5, 6]
[0, 4, 5, 7]
[0, 4, 6, 7]
[0, 5, 6, 7]
[1, 2, 3, 4]
[1, 2, 3, 5]
[1, 2, 3, 6]
[1, 2, 3, 7]
[1, 2, 4, 5]
[1, 2, 4, 6]
[1, 2, 4, 7]
[1, 2, 5, 6]
[1, 2, 5, 7]
[1, 2, 6, 7]
[1, 3, 4, 5]
[1, 3, 4, 6]
[1, 3, 4, 7]
[1, 3, 5, 6]
[1, 3, 5, 7]
[1, 3, 6, 7]
[1, 4, 5, 6]
[1, 4, 5, 7]
[1, 4, 6, 7]
[1, 5, 6, 7]
[2, 3, 4, 5]
[2, 3, 4, 6]
[2, 3, 4, 7]
[2, 3, 5, 6]
[2, 3, 5, 7]
[2, 3, 6, 7]
[2, 4, 5, 6]
[2, 4, 5, 7]
[2, 4, 6, 7]
[2, 5, 6, 7]
[3, 4, 5, 6]
[3, 4, 5, 7]
[3, 4, 6, 7]
[3, 5, 6, 7]
[4, 5, 6, 7]
ncomb = 70


In [28]:
next_comb(8,4,comb)

(0, [0, 1, 4, 6])